In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import curve_fit
import json

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import curve_fit


# 归一化函数
def normalize(tokens):
    scaler = MinMaxScaler(feature_range=(0, 100))
    tokens = np.array(tokens).reshape(-1, 1)
    return scaler.fit_transform(tokens).flatten()

# 三次函数模型
def cubic_model(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

In [ ]:
with open('../count_datas/tokens_train.json', 'r') as f:
    res_tokens = json.load(f)

In [ ]:
with open('../count_datas/tokens_infer.json', 'r') as f:
    res_tokens_infer = json.load(f)

In [ ]:
with open('../count_datas/f1.json', 'r') as f:
    res_f1 = json.load(f)

In [ ]:
def get_res(out_name, type_str, keys, train_keys, valid_key,fig_out_name):
    tmp_res = {}
    tmp_res['train'] = {}
    tmp_res['valid'] = {}
    for key in keys:
        now_tokens = res_tokens[key]
        now_tokens_infer = res_tokens_infer[key]
        now_f1 = res_f1[key]

        need_keys = [
            ['{}_bidirection_A','{}_pipeline_A',],
            ['{}_bidirection_B','{}_pipeline_B',],
            ['{}_bidirection_C','{}_pipeline_C',],
            ['{}_bidirection_D','{}_pipeline_D',],
            ['{}_bidirection_E','{}_pipeline_E',],
        ]
        processed_tmp_f1 = []
        processed_tmp_tokens = []
        processed_tmp_tokens_infer = []
        for need_key_list in need_keys:
            val_f1 = 0
            val_tokens = 0
            val_tokens_infer = 0
            for need_key in need_key_list:
                val_f1 += float(now_f1['{}_A'.format(need_key.format(type_str))][-1])
                val_tokens += now_tokens[need_key.format(type_str)]
                val_tokens_infer += now_tokens_infer[need_key.format(type_str)]
            processed_tmp_f1.append(val_f1/2)
            processed_tmp_tokens.append(val_tokens/2)
            processed_tmp_tokens_infer.append(val_tokens_infer/2)


        # print('处理前:{}'.format(processed_tmp_tokens))
        sum_val = 0
        for i in [1, 3, 4]:
            sum_val += processed_tmp_tokens_infer[i]
        # print('sum_val:{}'.format(sum_val))
        sum_val = sum_val/3
        # print('sum_val:{}'.format(sum_val))
        portions = []
        for i in [1, 3, 4]:
            portion = (processed_tmp_tokens_infer[i]/sum_val - 1)*1 +1
            portions.append(portion)
            processed_tmp_tokens[i] *= portion
        for i in range(len(processed_tmp_tokens)):
            processed_tmp_tokens[i] = int(processed_tmp_tokens[i])
        # 输出调整后的processed_tmp_tokens
        # print('portions:{}'.format(portions))
        # print(processed_tmp_tokens)

        y_ref = (float(now_f1['{}_v0_A'.format(type_str)][-1]) + float(now_f1['{}_pipeline_A_A'.format(type_str)][-1]) + float(now_f1['{}_bidirection_A_A'.format(type_str)][-1])) / 3
        # index_list = np.argsort(processed_tmp_tokens)
        index_list = [0,1,2,3,4]
        processed_tmp_tokens = [processed_tmp_tokens[i] for i in index_list]
        processed_tmp_f1 = [processed_tmp_f1[i] for i in index_list]
        if key in train_keys:
            tmp_res['train']['{}'.format(key)] = {
                'f1':processed_tmp_f1,
                'tokens':processed_tmp_tokens,
                'tokens_infer':processed_tmp_tokens_infer,
                'portions': portions,
                'y_ref': y_ref
            }
        else:
            tmp_res['valid']['{}'.format(key)] = {
                'f1':processed_tmp_f1,
                'tokens':processed_tmp_tokens,
                'tokens_infer':processed_tmp_tokens_infer,
                'portions': portions,
                'y_ref': y_ref
            }

    all_tokens = []
    all_f1 = []
    all_portions = []
    for key in train_keys:
        tokens = tmp_res['train']['{}'.format(key)]['tokens']
        y_ref = tmp_res['train']['{}'.format(key)]['y_ref']
        f1 = tmp_res['train']['{}'.format(key)]['f1']
        portions = tmp_res['train']['{}'.format(key)]['portions']
        all_portions.append(portions)
        # print(key)
        # print('tokens:{}'.format(tokens))
        # print('y_ref:{}'.format(y_ref))
        # print('f1:{}'.format(f1))
        # print('portions:{}'.format(portions))
        tokens_normalized = normalize(tokens)
        f1_adjusted = np.array(f1) - y_ref  # Adjust y by subtracting y_ref
        all_tokens.extend(tokens_normalized)
        all_f1.extend(f1_adjusted)

    all_tokens = np.array(all_tokens)
    all_f1 = np.array(all_f1)
    # 使用三次函数进行拟合
    popt_cubic, _ = curve_fit(cubic_model, all_tokens, all_f1, p0=(1, 1, 1, 0))

    # 绘制拟合曲线
    plt.figure(figsize=(10, 6))
    plt.scatter(all_tokens, all_f1, color='blue', label='Data')
    plt.plot(np.linspace(0, 100, 500), cubic_model(np.linspace(0, 100, 500), *popt_cubic), color='red', label='Fitted cubic curve')
    plt.title('Curve Fitting for F1 vs Tokens (Adjusted by y_ref) - Cubic Model')
    plt.xlabel('Normalized Tokens')
    plt.ylabel('Adjusted F1')
    plt.legend()
    plt.grid(True)
    plt.savefig('./fig_拟合算力/{}.png'.format(fig_out_name), dpi=300, bbox_inches='tight')
    plt.show()


    a,b,c,d = popt_cubic
    average_portions = [sum(x[i] for x in all_portions) / len(all_portions) for i in range(len(all_portions[0]))]
    average_portions = [1,average_portions[0],1,average_portions[1],average_portions[2]]
    valid_res = []
    valid_tokens = tmp_res['valid']['{}'.format(valid_key)]['tokens']
    valid_y_ref = tmp_res['valid']['{}'.format(valid_key)]['y_ref']

    valid_tokens_normalized = normalize(valid_tokens)
    print('valid_tokens_normalized:{}'.format(valid_tokens_normalized))
    print('valid_y_ref:{}'.format(valid_y_ref))
    for token in valid_tokens_normalized:
        print('token:{}'.format(token))
        print('cubic_model(token,a,b,c,d):{}'.format(cubic_model(token,a,b,c,d)))
        valid_res.append(cubic_model(token,a,b,c,d)+valid_y_ref)
    tmp_res['valid']['valid_res'] = valid_res
    tmp_res['train']['xishu'] = list(popt_cubic)
    print('valid_res:{}'.format(valid_res))
    print('原始valid_res:{}'.format(tmp_res['valid']['{}'.format(valid_key)]['f1']))
    return tmp_res

In [ ]:
res = {}
out_name = 'CMeIE'
type_str = 'withtype'
keys = [
    'CMeIE-glm4',
    'CMeIE-qwen2.5',
    'CMeIE-baichuan2',
    'CMeIE-intern2.5',
]
train_keys = [
    'CMeIE-glm4',
    'CMeIE-qwen2.5',
    'CMeIE-baichuan2',
]
valid_key = 'CMeIE-intern2.5'
fig_out_name = 'CMeIE---withtype'
tmp_res = get_res(out_name, type_str, keys, train_keys, valid_key, fig_out_name)
res['CMeIE---带实体类型'] = tmp_res

In [ ]:
out_name = 'CMeIE'
type_str = 'notype'
fig_out_name = 'CMeIE---notype'
tmp_res = get_res(out_name, type_str, keys, train_keys, valid_key, fig_out_name)
res['CMeIE---不带实体类型'] = tmp_res

In [ ]:
out_name = 'ADE'
type_str = 'withtype'
keys = [
    'ADE-glm4',
    'ADE-qwen2.5',
    'ADE-llama3.1',
    'ADE-deepseek',
]
train_keys = [
    'ADE-qwen2.5',
    'ADE-glm4',
    'ADE-deepseek',   
]
valid_key = 'ADE-llama3.1'
fig_out_name = 'ADE---withtype'
tmp_res = get_res(out_name, type_str, keys, train_keys, valid_key, fig_out_name)
res['ADE---带实体类型'] = tmp_res

In [ ]:
out_name = 'ADE'
type_str = 'notype'
fig_out_name = 'ADE---notype'
tmp_res = get_res(out_name, type_str, keys, train_keys, valid_key, fig_out_name)
res['ADE---不带实体类型'] = tmp_res

In [ ]:
with open('拟合算力_f1.json', 'w') as f:
    json.dump(res, f, ensure_ascii = False, indent=2)